```{=latex}
\usepackage{hyperref}
\usepackage{graphicx}
\usepackage{listings}
\usepackage{textcomp}
\usepackage{fancyvrb}

\newcommand{\passthrough}[1]{\lstset{mathescape=false}#1\lstset{mathescape=true}}
\newcommand{\tightlist}{}
```

```{=latex}
\title{Jupyter for DevOps}
\author{Moshe Zadka -- https://cobordism.com}
\date{}

\begin{document}
\begin{titlepage}
\maketitle
\end{titlepage}

\frame{\titlepage}
```

```{=latex}
\begin{frame}
\frametitle{Acknowledgement of Country}

Belmont (in San Francisco Bay Area Peninsula)

Ancestral homeland of the Ramaytush Ohlone people

\end{frame}
```

## SSH

### SSH with Paramiko

In [39]:
import paramiko
connect_params = dict(username="user", port=5022, look_for_keys=False, key_filename="build/client_rsa_key")

```{=latex}
\begin{frame}
\frametitle{Connect with Paramiko}[fragile]
```

In [45]:
client = paramiko.SSHClient()
client.set_missing_host_key_policy(
    paramiko.client.WarningPolicy)
client.connect("localhost", **connect_params)

```{=latex}
\end{frame}
```

### Run a command

```{=latex}
\begin{frame}
\frametitle{Run Command with Paramiko}[fragile]
```

In [45]:
res = client.exec_command("ls")
files = res[1].read().decode("ascii").splitlines()
files

['some_file', 'another_file']

```{=latex}
\end{frame}
```

### Loop

In [51]:
connect_params_list = [
    dict(username="user", port=5022, 
         look_for_keys=False, key_filename="build/client_rsa_key"),
    dict(username="user2", port=5022, 
        look_for_keys=False, key_filename="build/client_rsa_key")
]

```{=latex}
\begin{frame}
\frametitle{Automate with Paramiko}[fragile]
```

In [55]:
files = set()
for connect_params in connect_params_list:
    client.connect("localhost", **connect_params)
    res = client.exec_command("ls")
    files.update(res[1].read().decode("ascii").splitlines())
sorted(files)

['another_file', 'even_more_files', 'more_file', 'some_file']

```{=latex}
\end{frame}
```

## Cloud

### Configuring `boto3`

In [11]:
import boto3
import io
access_credentials = dict(
    endpoint_url='http://localhost:3000',
    aws_access_key_id="",
    aws_secret_access_key="",
)

```{=latex}
\begin{frame}
\frametitle{Connect to S3}[fragile]
```

In [12]:
s3 = boto3.client(
    service_name='s3',
    region_name='us-west-2',
    # Credentials can be read from
    # different sources.
    **access_credentials,
)

```{=latex}
\end{frame}
```

In [15]:
s3.create_bucket(Bucket="special-bucket.123.431")

{'ResponseMetadata': {'RequestId': 'JCNH811JQGW21OYQVQLZF9JUF9COHPYB5JFB78CQS817OI8LMCXX',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Werkzeug/2.1.2 Python/3.10.5',
   'date': 'Wed, 22 Jun 2022 01:48:31 GMT',
   'x-amzn-requestid': 'JCNH811JQGW21OYQVQLZF9JUF9COHPYB5JFB78CQS817OI8LMCXX',
   'content-type': 'text/html; charset=utf-8',
   'content-length': '176',
   'access-control-allow-origin': '*',
   'connection': 'close'},
  'RetryAttempts': 0}}

### Uploading an S3 file

```{=latex}
\begin{frame}
\frametitle{Upload to S3}[fragile]
```

In [16]:
some_contents = io.BytesIO(b"some contents")
s3.upload_fileobj(
    some_contents,
    "special-bucket.123.431",
    "some-contents.txt",
)

```{=latex}
\end{frame}
```

### Looping

```{=latex}
\begin{frame}
\frametitle{Automate Uploading to S3}[fragile]
```

In [17]:
for i in range(10):
    some_contents = io.BytesIO(f"some {i} contents".encode("ascii"))
    s3.upload_fileobj(
        some_contents,
        "special-bucket.123.431",
        f"some-contents-{i}.txt",
    )

```{=latex}
\end{frame}
```

## GitLab

In [40]:
import gitlab
import pathlib
import base64

token = (pathlib.Path.home() / ".gitlab-token").read_text().strip()
project_name = "moshez/project1"
projects = [project_name, "moshez/2019"]

### Configuring the client

```{=latex}
\begin{frame}
\frametitle{Configuring Gitlab}[fragile]
```

In [41]:
client = gitlab.Gitlab(private_token=token)

```{=latex}
\end{frame}
```

### Analyzing one project

```{=latex}
\begin{frame}
\frametitle{Analyzing README}[fragile]
```

In [42]:
project = client.projects.get(project_name)
[readme] = [
    obj
    for obj in project.repository_tree(as_list=False)
    if obj["name"] == "README.md"
]
contents = project.repository_blob(readme["id"])
data = base64.b64decode(contents["content"].encode("ascii")).decode("utf-8")
len(data.split())

882

```{=latex}
\end{frame}
```

### Looping

```{=latex}
\begin{frame}
\frametitle{Analyzing projects in a loop}[fragile]
```

In [38]:
for project_name in projects:
    project = client.projects.get(project_name)
    [readme] = [
        obj
        for obj in project.repository_tree(as_list=False)
        if obj["name"] == "README.md"
    ]
    contents = project.repository_blob(readme["id"])
    data = base64.b64decode(contents["content"].encode("ascii")).decode("utf-8")
    print(len(data.split()))

882
563


```{=latex}
\end{frame}
```

## Summary

```{=latex}
\begin{frame}
\frametitle{Why Jupyter for DevOps?}[fragile]

\pause
\begin{itemize}
\item Prototype \pause
\item Iterate \pause
\item Automate \pause
\item Document \pause
\item Share \pause
\end{itemize}


\end{frame}
```

* Iterate
* Document
* Share

```{=latex}
\end{document}
```